# RuleOne Financial Analysis Notebook

This notebook demonstrates querying SEC EDGAR data from SQLite and visualizing financial metrics using Plotly.NET.

In [ ]:
#r "nuget: Plotly.NET, 5.0.0"
#r "nuget: Plotly.NET.Interactive, 5.0.0"
#r "nuget: Microsoft.Data.Sqlite, 9.0.0"

open System
open Microsoft.Data.Sqlite
open Plotly.NET
open Plotly.NET.LayoutObjects

## Connect to Database

First, let's connect to our RuleOne SQLite database and query some facts.

In [ ]:
// Database connection
let dbPath = "ruleone.db"
let connectionString = $"Data Source={dbPath}"

// Helper function to query facts by concept
let queryFactsByConcept (concept: string) =
    use connection = new SqliteConnection(connectionString)
    connection.Open()
    
    let queryCommand = connection.CreateCommand()
    queryCommand.CommandText <- "SELECT * FROM Facts WHERE Concept LIKE @concept ORDER BY FilingDate ASC"
    queryCommand.Parameters.AddWithValue("@concept", $"%%{concept}%%") |> ignore
    
    use reader = queryCommand.ExecuteReader()
    
    let results = ResizeArray<_>()
    while reader.Read() do
        results.Add({|
            CIK = reader.GetString(1)
            CompanyName = if reader.IsDBNull(2) then "Unknown" else reader.GetString(2)
            FilingDate = if reader.IsDBNull(3) then "Unknown" else reader.GetString(3)
            Concept = reader.GetString(5)
            Value = if reader.IsDBNull(6) then "0" else reader.GetString(6)
            Period = if reader.IsDBNull(9) then "Unknown" else reader.GetString(9)
        |})
    
    connection.Close()
    results |> Seq.toList

printfn "Database helper functions loaded"

## Query Revenue Data

Let's query revenue data from our database.

In [ ]:
// Query revenue facts
let revenueFacts = queryFactsByConcept "Revenues"

printfn "Found %d revenue records" (List.length revenueFacts)

// Display first few records
revenueFacts 
|> List.truncate 5
|> List.iter (fun fact -> 
    printfn "%s - %s: $%s" fact.CompanyName fact.Period fact.Value)

## Visualize Revenue Trend

Create a line chart showing revenue growth over time.

In [ ]:
// Prepare data for charting
let revenueData = 
    revenueFacts
    |> List.map (fun fact -> 
        let value = 
            try 
                decimal fact.Value / 1_000_000m // Convert to millions
            with _ -> 0m
        (fact.Period, value))
    |> List.sortBy fst

let periods = revenueData |> List.map fst
let values = revenueData |> List.map snd

// Create line chart
let revenueChart =
    Chart.Line(
        x = periods,
        y = values,
        Name = "Revenue"
    )
    |> Chart.withXAxisStyle(Title = Title.init("Period"))
    |> Chart.withYAxisStyle(Title = Title.init("Revenue (Millions USD)"))
    |> Chart.withTitle("Revenue Growth Trend")
    |> Chart.withSize(1000, 600)

revenueChart

## Query Earnings Data

Let's query net income/earnings data.

In [ ]:
// Query earnings facts
let earningsFacts = queryFactsByConcept "NetIncome"

printfn "Found %d earnings records" (List.length earningsFacts)

// Display first few records
earningsFacts 
|> List.truncate 5
|> List.iter (fun fact -> 
    printfn "%s - %s: $%s" fact.CompanyName fact.Period fact.Value)

## Visualize Revenue and Earnings Together

Create a combined chart showing both revenue and earnings trends.

In [ ]:
// Prepare earnings data
let earningsData = 
    earningsFacts
    |> List.map (fun fact -> 
        let value = 
            try 
                decimal fact.Value / 1_000_000m // Convert to millions
            with _ -> 0m
        (fact.Period, value))
    |> List.sortBy fst

let earningsPeriods = earningsData |> List.map fst
let earningsValues = earningsData |> List.map snd

// Create combined chart
let combinedChart =
    [
        Chart.Line(x = periods, y = values, Name = "Revenue")
        Chart.Line(x = earningsPeriods, y = earningsValues, Name = "Net Income")
    ]
    |> Chart.combine
    |> Chart.withXAxisStyle(Title = Title.init("Period"))
    |> Chart.withYAxisStyle(Title = Title.init("Amount (Millions USD)"))
    |> Chart.withTitle("Revenue and Earnings Trend")
    |> Chart.withSize(1000, 600)

combinedChart

## Calculate Growth Metrics

Use the RuleOne Analytics library to calculate CAGR (Compound Annual Growth Rate).

In [ ]:
// Load RuleOne Analytics library
// Assuming the library is built and accessible
// #r "../../src/RuleOne.Analytics/bin/Debug/net8.0/RuleOne.Analytics.dll"
// open RuleOne.Analytics.FinancialMetrics

// Manual CAGR calculation for demonstration
let calculateCAGR beginValue endValue years =
    if beginValue <= 0m || endValue <= 0m || years <= 0m then
        None
    else
        let ratio = endValue / beginValue
        let exponent = 1.0 / (float years)
        let cagr = (float ratio) ** exponent - 1.0
        Some (cagr * 100.0) // Return as percentage

// Calculate revenue CAGR if we have enough data
if List.length revenueData >= 2 then
    let firstRevenue = revenueData |> List.head |> snd
    let lastRevenue = revenueData |> List.last |> snd
    let years = decimal (List.length revenueData - 1)
    
    match calculateCAGR firstRevenue lastRevenue years with
    | Some cagr -> printfn "Revenue CAGR: %.2f%%" cagr
    | None -> printfn "Unable to calculate CAGR"
else
    printfn "Not enough data to calculate CAGR"

## Summary

This notebook demonstrates:
- Connecting to the RuleOne SQLite database
- Querying financial facts (revenue, earnings)
- Visualizing trends using Plotly.NET
- Calculating growth metrics (CAGR)

To use this notebook:
1. First run the ETL app to fetch and store SEC data: `dotnet run --project src/RuleOne.ETL 0000789019 10-K`
2. Open this notebook in Jupyter or Visual Studio Code with .NET Interactive
3. Execute cells sequentially to analyze the data